In [75]:
#import ee
#ee.Authenticate()

In [18]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np

import openpyxl

In [25]:
ee.Initialize()

In [26]:
ruta_catastro='projects/ee-bismarksr17/assets/CATASTRO_2023-02-25'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI_01_2023'

CODIGO_PROPIEDAD='unidad_01'
NOM_PROPIEDAD='unidad_02'
CODIGO_CANHERO='unidad_03'
NOM_CANHERO='unidad_04'

VARIEDAD='variedad'
ESTADO='RENOVACION'
SUPERFICIE='area'

In [27]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [28]:
#lista_props = [300, 1729, 1454, 30, 325, 17, 629, 19, 275, 256, 1728, 114, 949, 257, 29, 551, 31, 221, 92, 47, 478, 1618, 10, 1727, 700, 123, 259, 297, 591, 1314, 365, 239, 1450, 201, 674, 122, 216, 205, 495, 217, 375, 5, 33, 895, 63, 658, 23, 50, 1528, 1831, 185, 606, 343, 326, 510, 74, 132, 1011, 333, 327, 242, 299, 98, 1704, 294, 113, 560, 1215, 55, 306, 277, 752, 197, 209]
lista_props = [1831]

In [29]:
df_intersects = None

In [30]:
# Cargar el archivo de Excel existente
wb = openpyxl.load_workbook('DATA_EST.xlsx')
# Seleccionar la hoja de trabajo
ws = wb['data']

contador = 0
for cod_prop in lista_props:
    print('inicio:', cod_prop)
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad.filter(ee.Filter.neq(VARIEDAD, ESTADO))
    lotes_renov = propiedad.filter(ee.Filter.eq(VARIEDAD, ESTADO))
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_geopandas(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    
    intersect = overlay(lotes_local, vector_local, how="intersection")
    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    area_diff = (area_02 - area_01)/len(intersect)
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3], dina2.iloc[i,4], dina2.iloc[i,5], dina2.iloc[i,6]]
        ws.append(new_row)
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    
    contador = contador + 1 
    print('fin: ', cod_prop)
    print('___________________CONTADOR: ', contador)
wb.save('DATA_EST.xlsx')
print('FIN......!')

inicio: 1831


ValueError: Number of class have to be an integer greater than or equal to 1 and smaller than or equal to the number of unique values to use

In [17]:
df_intersects.head()

,area,cultivo,financia,fs,id,soca,textura,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,zona,count,label,geometry,area_2
0,8.11,canha,PROPIO,16/4/2021,00007520000L3.2,1,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.2,UCG_90-20,152,5,7,"POLYGON ((482296.408 8104190.000, 482295.764 8...",0.008011
1,14.28,canha,PROPIO,15/4/2020,00007520000L3.1,2,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.1,UCG_90-20,152,5,7,"POLYGON ((482295.764 8104220.000, 482296.408 8...",0.034528
2,8.11,canha,PROPIO,16/4/2021,00007520000L3.2,1,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.2,UCG_90-20,152,24,5,"POLYGON ((482300.000 8104022.518, 482299.840 8...",-0.003671
3,14.28,canha,PROPIO,15/4/2020,00007520000L3.1,2,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.1,UCG_90-20,152,24,5,"POLYGON ((482255.560 8103940.000, 482220.000 8...",0.222638
4,8.11,canha,PROPIO,16/4/2021,00007520000L3.2,1,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.2,UCG_90-20,152,2,6,"POLYGON ((482299.840 8104030.000, 482299.411 8...",-0.002981


In [16]:
df_intersects.to_file("INTERSECT.shp", driver="ESRI Shapefile")

In [91]:
Map = geemap.Map(basemap="SATELLITE")

In [92]:
Map.centerObject(propiedad.geometry(), 14)
Map.addLayer(lotes_canha.style(**vis_params_caña), {}, 'Lotes Caña')
Map.addLayer(lotes_renov.style(**vis_params_renov), {}, 'Lotes Renovación')

In [93]:
Map

Map(center=[-17.227364680451014, -62.98377937612558], controls=(WidgetControl(options=['position', 'transparen…

In [124]:
categoria = ['Nulo','Muy Bajo','Bajo Medio','Bajo','Medio','Medio Alto','Alto','Muy Alto']
colores = ['','','','','','','','']